In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates




In [ ]:
proton_events = {}
k = 1
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=10)
event_iterator = iter(source)
arr_max = []

for event in event_iterator:
    for tel in event.trigger.tels_with_trigger:
        print(event.r1.tel[tel].waveform.shape)
        #for pixel in event.r1.tel[1].waveform:
        #    arr_max.append(np.max(pixel))

In [ ]:
source.subarray.peek()

In [ ]:
tel_number

In [ ]:
#filename = '/Users/vdk/MST_work/data_muons/mono_proton_simtel/proton_20deg_0deg_run2___cta-prod6-paranal-2147m-Paranal-nshow-3000-0-LSTs-1-MSTs-0-SSTs-NSCAT_5_CSCAT_300_EMIN_1_EMAX_200_after_muontag.simtel.zst'
#filename = '/Users/vdk/MST_work/data_muons/mono_muon_simtel/muon_20deg_0deg_run0___cta-prod6-paranal-2147m-Paranal-0-LSTs-1-MSTs-0-SSTs-NSCAT_1_CSCAT_8_EMIN_0.005_EMAX_1_FIXCHI_600_VIEWCONE_3_after_muontag.simtel.zst'
#filename = '/Users/vdk/sim_run_folder/test321.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)



for i,j in enumerate(event_iterator):
    if i == 1:
        telescope = 1
        event = j
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[telescope].camera.geometry
        title="TrueImage run {} if {} fit {} width {}".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[telescope].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        for pix_id in range(0,1764):
            bx.plot(j.r0.tel[telescope].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
        bx.set_ylabel("ADC counts")
        bx.set_xlabel("Samples")
        bx.set_title("Waveforms")
        plt.show()
        #plt.savefig(f"/Users/vdk/MST_ProtonImages_NewQualityCheck/TrueImage+Waveform_mstPROTON_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        break
        #if i == 150:
        #    break

plt.close()

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    print(i, j.trigger.tels_with_trigger)
    

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

In [ ]:
event = next(event_iterator)

In [ ]:
for i in range(len(event.trigger.tels_with_trigger)):
    print(event.trigger.tels_with_trigger[i])




In [ ]:
camgeom = source.subarray.tel[1].camera.geometry

In [ ]:
disp = CameraDisplay(camgeom)
disp.image = next(event_iterator).simulation.tel[3].true_image

In [ ]:
event = next(event_iterator)

In [ ]:
print(len(event.simulation.tel[10].true_image))

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

events = [copy.deepcopy(event) for event in source]


for i,j in enumerate(event_iterator):
    print(f"Telescope with data: {event.r1.tel.keys()} in the events number {i}")

In [ ]:
obstime = Time("2013-11-01T03:00")
location = EarthLocation.of_site("Roque de los Muchachos")

altaz = AltAz(location=location, obstime=obstime)

array_pointing = SkyCoord(
    alt=event.pointing.array_azimuth,
    az=event.pointing.array_altitude,
    frame=altaz,
)

print(array_pointing)

In [ ]:
filename = '/Users/vdk/sim_run_folder/corsika_theta_28.4_az_182.75_run2.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

events = [copy.deepcopy(event) for event in source]
events[4]

In [ ]:
events[12].trigger.tel[3]

In [ ]:
stereo_mask = []
for i in events:
    check_LST1 = 1 in i.trigger.tels_with_trigger
    check_LST2 = 2 in i.trigger.tels_with_trigger
    if check_LST1 and check_LST2:
        stereo_mask.append(True)
        print(True)
    else:
        stereo_mask.append(False)        
        print(False)

In [ ]:
stereo_events = []
for i,j in enumerate(events):
    if stereo_mask[i]:
        stereo_events.append(j)
        print(j.trigger.tels_with_trigger)


In [ ]:
#for i in stereo_events:
    #print(i.trigger.tels_with_trigger)
len(stereo_events)

In [ ]:
source.subarray.peek()

In [ ]:
telescope = 1
camgeom = source.subarray.tel[telescope].camera.geometry
disp = CameraDisplay(camgeom)
disp.image = events[12].simulation.tel[telescope].true_image #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()

In [ ]:
telescope = 3
camgeom = source.subarray.tel[telescope].camera.geometry
disp = CameraDisplay(camgeom)
disp.image = events[6].simulation.tel[telescope].true_image #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()

In [ ]:
stereo_events[2].trigger.tels_with_trigger

In [ ]:
filename = '/Users/vdk/sim_run_folder/test123.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

for i,j in enumerate(event_iterator):
        telescope = 1
        event = j
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[telescope].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = bx)
        disp.image = j.simulation.tel[2].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #bx.set_ylabel("ADC counts")
        #bx.set_xlabel("")
        bx.set_title("LST2")
        #plt.show()
        plt.savefig(f"/Users/vdk/StereoProton_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        #if i == 150:
        #    break

plt.close()

In [ ]:
# pipeline to extract only stereo trigger for 2 LSTs
filename = '/Users/vdk/sim_run_folder/run1_nt2_mt2_at1.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

events = [copy.deepcopy(event) for event in source]

stereo_mask = []
for i in events:
    check_LST1 = 1 in i.trigger.tels_with_trigger
    check_LST2 = 2 in i.trigger.tels_with_trigger
    if check_LST1 and check_LST2:
        stereo_mask.append(True)
        #print(True)
    else:
        stereo_mask.append(False)        
        #print(False)

stereo_events = []
for i,j in enumerate(events):
    if stereo_mask[i]:
        stereo_events.append(j)
        #print(j.trigger.tels_with_trigger)

print("events = ", len(events))
print("stereo events = ", len(stereo_events))
source.subarray.peek()

In [ ]:
for i,j in enumerate(stereo_events):
        telescope = 1
        event = j
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[telescope].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = bx)
        disp.image = j.simulation.tel[2].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #bx.set_ylabel("ADC counts")
        #bx.set_xlabel("")
        bx.set_title("LST2")
        #plt.show()
        plt.savefig(f"/Users/vdk/StereoProtonNT2MT2_AT1_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        #if i == 150:
        #    break

plt.close()

In [ ]:
for i in stereo_events:
    print(i.trigger.tels_with_trigger)

In [ ]:
source.subarray.tels

In [ ]:
# pipeline to extract only stereo trigger for 2 LSTs
all_events = []
for i in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/numtele2_maxtele2_arraytelescope1/corsika_theta_28.4_az_182.75_run{i}.simtel.gz'
    source = EventSource(filename, max_events=10000)
    events = [copy.deepcopy(event) for event in source]
    all_events.append(events)

#stereo_mask = []
#for i in events:
#    check_LST1 = 1 in i.trigger.tels_with_trigger
#    check_LST2 = 2 in i.trigger.tels_with_trigger
#    if check_LST1 and check_LST2:
#        stereo_mask.append(True)
#        #print(True)
#    else:
#        stereo_mask.append(False)        
#        #print(False)

#stereo_events = []
#for i,j in enumerate(events):
#    if stereo_mask[i]:
#        stereo_events.append(j)
#        #print(j.trigger.tels_with_trigger)

#print("events = ", len(events))
#print("stereo events = ", len(stereo_events))
#source.subarray.peek()



In [ ]:
full_stereo_events = []
count = 0
for j in range(1,40):
    for i in all_events[j]:
        count += 1
        if len(i.trigger.tels_with_trigger) > 1:
            full_stereo_events.append(i)

In [ ]:
len(full_stereo_events)

In [ ]:
count

In [ ]:
plt.figure(figsize = (9,6))
#camgeom = source.subarray.tel[m].camera.geometry
#title=f"1"
#title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
#disp = CameraDisplay(camgeom,title=title)
#disp.image = event_container[2414].r1.tel[1].waveform.sum(axis=1)
#disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
#disp.cmap = plt.cm.RdBu_r
#disp.add_colorbar()
#disp.set_limits_percent(95)
plt.grid(alpha = 0.6)
#plt.yticks([-0.5,0,0.5,1,1.1,1.2,1.3,1.4])
for pix_id in range(0,1855):
    plt.plot(event_container[2414].r1.tel[1].waveform[pix_id], drawstyle='steps')

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.4
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1:            
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
            calib(j)
            image_processor(j)
            muon_processor(j)
            for m in j.muon.tel.keys():
                if not np.isnan(j.muon.tel[m].efficiency.width):
                    event_container[keks+i] = j
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
### NEW CLEANING NEW CLEANING NEW CLEANING
####
###
###
# Algorithm to process events and calibrate with new cleaning algo
###
int_cut = 0.2
min_pixel = 150

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
cleaned_event_container = {}
t1 = time.perf_counter(), time.process_time()
for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1: 
            event = j
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            for tels in event.trigger.tels_with_trigger:
                pixel_mask = []
                for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                    if np.argmax(l) in range(16,40): 
                        pixel_mask.append(True)
                    else:
                        pixel_mask.append(False)
                working_wave = event.r1.tel[tels].waveform.copy()
                for k,l in enumerate(working_wave):
                    if pixel_mask[k]:
                        #working_wave[k] = np.zeros(40)     
                        working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                event.r1.tel[tels].waveform = working_wave
            
                calib(event)
                image_processor(event)
                muon_processor(event)
        for m in j.muon.tel.keys():
            if not np.isnan(j.muon.tel[m].efficiency.width):
                cleaned_event_container[keks+i] = j
        t2 = time.perf_counter(), time.process_time()
        




            
            
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")      


In [ ]:
### NEW CLEANING NEW CLEANING NEW CLEANING ONLY FOR THE EVENTS WHICH WAS ALREADY FITTED
####
###
###
# Algorithm to process events and calibrate with new cleaning algo
###
int_cut = 0.3
min_pixel = 150

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
cleaned_event_container = {}
t1 = time.perf_counter(), time.process_time()
for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        #print("new event")
        #checker += 1
        #if len(j.trigger.tels_with_trigger) > 1: 
        
        #if keks+i in event_container.keys(): # Thats for events that already was fitted
        
        #print(keks+i, "I AM HERE!!!")
        event = j
        image_processor = ImageProcessor(source.subarray)
        #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)



        for tels in event.trigger.tels_with_trigger:
            pixel_mask = []
            original_r1_waveform = event.r1.tel[tels].waveform.copy()
            for_clean_mask = np.array([]).astype(bool)

            for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                if np.argmax(l) in range(17,40): 
                    pixel_mask.append(True)
                else:
                    pixel_mask.append(False)
            working_wave = event.r1.tel[tels].waveform.copy()
            for k,l in enumerate(working_wave):
                if pixel_mask[k]:
                    for_clean_mask = np.append(for_clean_mask, True)
                    working_wave[k] = np.zeros(40)     
                    #working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                else:
                    for_clean_mask = np.append(for_clean_mask, False)
            event.r1.tel[tels].waveform = working_wave

            calib(event)
            image_processor(event)
            cam_geometry = source.subarray.tel[tels].camera.geometry

        return_pixels = np.array([])

        n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)

        for t in range(1,n_islands+1):
            if len(np.where(island_id == t)[0]) <= 3:
                return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])

        return_pixels = return_pixels.astype(int)


        event.r1.tel[tels].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]  

        calib(event)
        image_processor(event)
        muon_processor(event)   
                
        if len(j.muon.tel.keys()):       
            for m in j.muon.tel.keys():
                if not np.isnan(j.muon.tel[m].efficiency.width):
                    cleaned_event_container[keks+i] = j
        t2 = time.perf_counter(), time.process_time()
        




            
            
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")      



In [ ]:
print("fitted events - ", len(event_container))
print("fitted cleaned events - ", len(cleaned_event_container))
print("overall dataset - ", checker)

In [ ]:
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        print("m", m)
        print(event_container[i].index.event_id, "= ", event_container[i].muon.tel[m].efficiency.optical_efficiency)

In [ ]:
optical_coef

In [ ]:
optical_coef

In [ ]:
#
#
# Need this to fill optical efficiency array with new values
#
#
optical_coef = []
event_optical_coef = []
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        #print(np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency))
        #if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.3 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5:
        if not np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency) and event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50:
            optical_coef.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)
            event_optical_coef.append(i)

print(np.mean(optical_coef))
print(np.std(optical_coef))
print(np.std(optical_coef)/np.mean(optical_coef))
print(len(optical_coef))
#optical_coef = optical_coef[0:31]

In [ ]:
print(res_dict)
print(res_dict_cleaned)

In [ ]:
#
#
# Code for extracting events which are presented in both cleaned and raw dataset
#
#

# dictionary with event:opticalefficiency for original events
res_dict = {event_optical_coef[i]:optical_coef[i] for i in range(len(optical_coef))}
# dictionary with event:opticalefficiency for cleaned events
res_dict_cleaned = {events_opt_eff_cleaned[i]:opt_eff_cleaned[i] for i in range(len(opt_eff_cleaned))}
print(len(res_dict))
print(len(res_dict_cleaned))

plot = []
for i in res_dict_cleaned.keys():
    if i in res_dict.keys():
        plot.append(res_dict[i])
    else:
        plot.append(0)

new_plot = []
new_ind = []
for i,j in enumerate(plot):
    if not j == 0:
        new_plot.append(j)
        new_ind.append(i)


        
print("len of remained = ", len(new_plot))
print("len of all = ", len(opt_eff_cleaned))

cumulative_dif = 0

for i in res_dict:
    if i in res_dict_cleaned:
        cumulative_dif += abs(res_dict[i] - res_dict_cleaned[i])

print("cumulative_dif = ", cumulative_dif)

print("relative cumulative_dif = ", cumulative_dif/len(new_plot))

In [ ]:
#
#
# Images for events without cleaning
#
#




x_range_1 = [i for i in range(len(optical_coef))]
#std_arr = [np.std(optical_coef)/np.mean(optical_coef) for i,j in enumerate(optical_coef)]
std_arr_1 = [np.std(optical_coef)/i**0.5 for i,j in enumerate(optical_coef)]
std_arr_1_big = [np.std(optical_coef) for i,j in enumerate(optical_coef)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


mean_range_1 = [i-1 for i in range(len(optical_coef)+3)]
y_upper_1 = [np.mean(optical_coef)+np.std(optical_coef) for i in mean_range_1]
y_lower_1 = [np.mean(optical_coef)-np.std(optical_coef) for i in mean_range_1]
y_mid_1 = [np.mean(optical_coef) for i in mean_range_1]
y_mid_1 = [np.mean(optical_coef[:i+1]) for i in range(len(optical_coef[:i+1]))]


#y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]

y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]


#y_upper_1 = [j+np.std(optical_coef[:i+1]) for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-np.std(optical_coef[:i+1]) for i,j in enumerate(y_mid_1)]


y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate([np.mean(optical_coef) for i in x_range_1])]
y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate([np.mean(optical_coef) for i in x_range_1])]

plt.figure(figsize = (12,6))
plt.ylim(0.095,0.34)

plt.style.use('default')
plt.xlim(-1,len(optical_coef))
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency with std")
#plt.title("Opt eff for muon rings without performing additional cleaning int_cut<0.1")
plt.title("Events before additional cleaning (pix > 50, intensity_cut = 0.3, completeness>0.5)")
plt.grid(alpha=0.5)
#plt.ylim(0,0.4)

plt.scatter(x_range_1, optical_coef)
plt.errorbar(x_range_1, optical_coef, yerr=std_arr_1_big, fmt='o', ms = 7, alpha = 0.8, color = 'darkorange')#, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
#plt.plot(x_range_1, y_mid_1, label = "mean", c = 'green', linewidth = 1.5)
plt.plot(x_range_1, [np.mean(optical_coef) for i in x_range_1], label = "mean", c = 'green', linewidth = 1.5)


plt.legend()
print("mean = ", np.mean(optical_coef))
print("std = ", np.std(optical_coef))
print("rel_err = ", np.std(optical_coef)/np.mean(optical_coef))
print("len = ", len(optical_coef))
#plt.savefig("/Users/vdk/ImagePurgatorium/LSTprotonsWithoutCleaningIntCut30%_Containment>50%_v2.png")

In [ ]:
# 0-?, 1 - 17, 2 - 20, 3-22, 4 - 45 for cleaned intensity check < 0.1 and original intensity  check < 0.2
# 0-5, 1-8, 2 - ?, 3 - ?, 4 - 17, 5 - 18, 6-19, 7-?, 8-20, 9-21, 10-22, 11-28, 12-?, 13-?, 14-?,15-?, 16-37,17-38
count = 0
#indices = [17,20,25,37]

new_indices = []
val_indices = []
std_arr_2 = []
for i,j in enumerate(optical_coef):
    if i in indices:
        count += 1
        val_indices.append(j)
        std_arr_2.append(std_arr_1[i])
        new_indices.append(count)



print(len(val_indices))
print(len(new_indices))
count = 0

for i,j in enumerated(event_optical_coef):
    if j in events_opt_eff_cleaned:
        new_indices.append(count)
        val_indices.append()
        count += 1

In [ ]:
opt_eff_cleaned

In [ ]:
# opt_eff_cleaned[11] = 0.2 + 0.14
# opt_eff_cleaned[12] = 0.17 + 0.08
# opt_eff_cleaned[-5] = 0.17 - 0.03
# opt_eff_cleaned[-6] = 0.16 - 0.04
# for i in range(3,7):
#     opt_eff_cleaned[i] = opt_eff_cleaned[i] + (-0.01*i)**i
# for i in range(16,21):
#     opt_eff_cleaned[i] = opt_eff_cleaned[i] + (-0.02*i)**i
# for i in range(44,48):
#     opt_eff_cleaned[i] = opt_eff_cleaned[i] + (-0.02*i)**i   

In [ ]:
#
#
# Images for events after cleaning
#
#

#plt.style.use('tableau-colorblind10')
plt.style.use('default')


#std_arr_3 = []

#for i,j in enumerate(optical_coef):
#    std_arr_3.append(np.std(optical_coef[:i+1])/(i**0.5))

#print(len(new_ind), len(std_arr_3))   

#std_arr = []

#for i,j in enumerate(opt_eff_cleaned):
#    std_arr.append(np.std(opt_eff_cleaned[:i+1])/(i**0.5))
    #print(np.std(opt_eff_cleaned[:i+2]))
    #print(len(opt_eff_cleaned[:i+1]))


mean_range = [i-1 for i in range(len(opt_eff_cleaned)+2)]
mean_range = [i-1 for i in range(len(opt_eff_cleaned)+2)]

y_mid = [np.mean(opt_eff_cleaned) for i in mean_range]
y_mid = [np.mean(opt_eff_cleaned[:i+1]) for i in range(len(opt_eff_cleaned[:i+1]))]


y_upper = [np.mean(opt_eff_cleaned)+np.std(opt_eff_cleaned) for i in mean_range]
y_lower = [np.mean(opt_eff_cleaned)-np.std(opt_eff_cleaned) for i in mean_range]



x_range = [i for i in range(len(opt_eff_cleaned))]




#std_arr = [np.std(optical_coef)/np.mean(optical_coef) for i,j in enumerate(optical_coef)]
std_arr = [np.std(opt_eff_cleaned)/i**0.5 for i,j in enumerate(opt_eff_cleaned)]
std_arr_big = [np.std(opt_eff_cleaned) for i,j in enumerate(opt_eff_cleaned)]
std_arr_3 = [np.std(new_plot) for i,j in enumerate(new_plot)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


plt.figure(figsize = (12,6))
plt.ylim(0.095,0.34)
#plt.ylim(0.095,0.265)
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency value")
#plt.title("Opt eff for muon rings after time sample cleaning, pix > 50, intensity<0.2")
plt.title("Events after additional cleaning (pix > 50, intensity_cut = 0.3, time_bin<16, completeness>0.5)")
#plt.title("Events after island cleaning (size $\leq 4$)")


plt.grid(alpha=0.5)
#plt.ylim(0.05,0.3)
#plt.scatter(new_ind, new_plot, color = 'orange')


y_upper = [j+std_arr[-1] for i,j in enumerate(y_mid)]
y_lower = [j-std_arr[-1] for i,j in enumerate(y_mid)]


y_upper = [j+std_arr[-1] for i,j in enumerate([np.mean(opt_eff_cleaned) for i in x_range])]
y_lower = [j-std_arr[-1] for i,j in enumerate([np.mean(opt_eff_cleaned) for i in x_range])]

plt.errorbar(x_range, opt_eff_cleaned, yerr=std_arr_big, fmt='o', ms = 5, alpha = 0.8, label="cleaned")#, c = 'blue')#, s = 100)#, c= 'black')
plt.errorbar(new_ind, new_plot, yerr=std_arr_3, fmt='o', ms = 5, alpha = 0.8, label="original") #, c = 'blue')#, s = 100)#, c= 'black')
#plt.scatter(new_ind, new_plot, alpha = 0.9, color = "orange", label="original") #, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(mean_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
plt.fill_between(x_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
#plt.plot(mean_range, y_mid, label = "mean", lw = 1.5, color='green')
#plt.plot(x_range, y_mid, label = "mean", lw = 1.5, color='green')
plt.plot(x_range, [np.mean(opt_eff_cleaned) for i in x_range], label = "mean", lw = 1.5, color='green')

#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'r', alpha = 0.3, label = 'mean +/- $1\sigma$')

plt.xlim(-1,len(opt_eff_cleaned))
plt.legend()
print("mean = ", np.mean(opt_eff_cleaned))
print("std = ", np.std(opt_eff_cleaned))
print("rel_err = ", np.std(opt_eff_cleaned)/np.mean(opt_eff_cleaned))
print("len of all = ", len(opt_eff_cleaned))
print("len of remained = ", len(new_plot))
#plt.savefig("/Users/vdk/ImagePurgatorium/CleanedOptEff_pix>50_int=01_16timebin_complet>05_RelStdErr.png")
#plt.savefig("/Users/vdk/ImagePurgatorium/IslandsCleaned_maskMEAN_Island4Pixel.png", dpi=900)

In [ ]:
new_plot

In [ ]:
opt_eff_cleaned

In [ ]:
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[tel_nummer].camera.geometry
title="Muon Ring?"
disp = CameraDisplay(camgeom,title=title)
disp.image = event_container[event_nummer].simulation.tel[tel_nummer].true_image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
#plt.savefig(f"/Users/vdk/IntensityCut_event{j.index.event_id}")

In [ ]:
# PROCESSING OF REGULAR IMAGES
lets_check = 0
opt_eff_cleaned = []
reg_event_number = []
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("MSE ", event_container[i].muon.tel[m].parameters.mean_squared_error)                
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                #if event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.1:
                #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                lets_check+=1
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"Event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event_container[i].simulation.tel[m].true_image
                #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                reg_event_number.append(i)
                plt.show()
                #plt.savefig(f"/Users/vdk/ImagePurgatorium/MuonsWithoutCleaning_intensitycut<0.3_event{i}.png")
                #opt_eff_cleaned.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)
                    
plt.close()

In [ ]:
print(reg_event_number)

In [ ]:
# FIND DIFFERENCE IN EVENTS FROM DIFFERENT CUTS

set1 = [2414, 3211, 3403, 4426, 6200, 6317, 6466, 7418, 8234, 8448, 9404, 9433, 12262, 12334, 14335, 14402, 15185, 15405, 19103, 20229, 20402, 21288, 21433, 22053, 22205, 22271, 22390, 23382, 24482, 26113, 26164, 26347, 26365, 27176, 27374, 28000, 28034, 28328, 29266, 29390, 30405, 30462, 31176, 31386, 32064, 32298, 33057, 33071, 35052, 35219, 35375, 35435, 36231, 36364, 36452, 37363, 39094, 39152, 39376]


set_dif = set(reg_event_number).difference(set(set1))
set_dif = [3198, 3403, 6317, 6466, 8234, 8448, 9404, 9433, 12262, 14402, 14406, 16110, 16379, 19103, 20229, 20402, 21288, 21433, 22123, 22205, 22364, 22365, 22390, 22415, 25051, 25345, 26113, 26164, 26347, 26365, 27176, 27374, 28000, 28328, 29266, 29390, 31176, 31386, 32064, 32298, 33057, 33071, 34115, 35052, 35219, 35375, 35435, 36231, 36452]
for i in set_dif:
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("MSE ", event_container[i].muon.tel[m].parameters.mean_squared_error)                
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                #if event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.1:
                #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                lets_check+=1
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"Event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event_container[i].simulation.tel[m].true_image
                #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                reg_event_number.append(i)
                plt.show()

In [ ]:
print(event_container[22415].muon.tel[1].efficiency)

In [ ]:
# PROCESSING OF REGULAR IMAGES 
##
#
# For all telescopes
#
#
#

lets_check = 0
opt_eff_cleaned = []
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        #if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                print("len from r1 ", len((np.where(event_container[i].r1.tel[m].waveform > 2)[0])))
                #print("MSE ", event_container[i].muon.tel[m].parameters.mean_squared_error)                
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                #if event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.1:
                #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                lets_check+=1
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"Event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event_container[i].simulation.tel[m].true_image
                #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                #plt.show()
                #plt.savefig(f"/Users/vdk/ImagePurgatorium/MuonsWithoutCleaning_intensitycut<0.3_event{i}.png")
                #opt_eff_cleaned.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)
                    
plt.close()

In [ ]:
# PROCESSING OF CLEANED IMAGES
lets_check = 0
opt_eff_cleaned = []
events_opt_eff_cleaned = []

r1_waves = []
n_pixel = []
energies = []
int_ratio = []
clean_event_number = []

for i in cleaned_event_container:
    for m in cleaned_event_container[i].trigger.tel.keys():
        if not np.isnan(cleaned_event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.7 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5 and cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error.to_value() < 0.01:
                #if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 30:
                    #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                    #print("n_pixels ", cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                    #print("r1 waveform", len(np.where(cleaned_event_container[i].r1.tel[m].waveform > 2)[0]))
                    r1_waves.append(len(np.where(cleaned_event_container[i].r1.tel[m].waveform > 2)[0]))
                    n_pixel.append(cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                    energies.append(cleaned_event_container[i].simulation.shower.energy)
                    int_ratio.append(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio)
                    #print("len from r1 ", len((np.where(cleaned_event_container[i].r1.tel[m].waveform > 2)[0])))
                    #print("MSE ", cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error)                
                    #print("Muon parameters ", cleaned_event_container[i].muon.tel[m].parameters)   
                    #print("intensity_check ", abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                    #print("Muon parameters ", cleaned_event_container[i].muon.tel[m].parameters)   
                    #print("intensity_check ", abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                    lets_check+=1
                    plt.figure(figsize = (9,6))
                    camgeom = source.subarray.tel[m].camera.geometry
                    #title=f"event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                    title=f"CleanedRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                    #title=f"TrueImage for events tagged as muon ring by cleaning algorithm {i}"
                    disp = CameraDisplay(camgeom,title=title)
                    disp.image = cleaned_event_container[i].simulation.tel[m].true_image
                    #disp.image = cleaned_event_container[i].r1.tel[m].waveform.sum(axis=1)
                    #disp.image = cleaned_event_container[i].simulation.tel[m].true_image
                    disp.cmap = plt.cm.RdBu_r
                    disp.add_colorbar()
                    disp.set_limits_percent(95)
                    clean_event_number.append(i)
                    plt.show()
                    #plt.savefig(f"/Users/vdk/ImagePurgatorium/TrueImagesOfCleanedMuons_intensitycut<0.3_event{i}.png")
                    opt_eff_cleaned.append(cleaned_event_container[i].muon.tel[m].efficiency.optical_efficiency)
                    events_opt_eff_cleaned.append(i)
                    
plt.close()

In [ ]:
print(clean_event_number)
print(len(clean_event_number))

In [ ]:
ranger = [i for i in range(len(energies))]
energies_new = [i.to_value() for i in energies]
histplot(energies_new, log_scale = True, bins = 10)
plt.xlabel("Energy [TeV]")
plt.ylabel("Number of muon rings")
plt.xlim(0,10)
plt.title("Energy spectrum of muon rings")
#plt.savefig('/Users/vdk/EnergyMuons.png')

In [ ]:
ranger = [i for i in range(len(n_pixel))]
x,y = np.histogram(int_ratio)
histplot(int_ratio, bins = 7)
#plt.xlim(0,150)
plt.xlabel("Intensity ratio")
plt.ylabel("Number of muon rings")
plt.title("Ratio of intensity inside muon ring to overall intensity")
plt.savefig('/Users/vdk/IntRtio.png')

In [ ]:
ranger = [i for i in range(len(n_pixel))]
x,y = np.histogram(n_pixel)
histplot(n_pixel, bins = 5)
plt.xlim(0,150)
plt.xlabel("Number of pixels in the ring")
plt.ylabel("Number of muon rings")
plt.savefig('/Users/vdk/NumberPixelHist.png')

In [ ]:
plt.plot(y[:-1],x)

In [ ]:
x_wave_range = [i for i in range(len(r1_waves))]

r1_waves = np.array(r1_waves)
n_pixel = np.array(n_pixel)

plt.plot(x_wave_range, r1_waves)
plt.plot(x_wave_range, n_pixel)
plt.plot(x_wave_range, r1_waves-n_pixel)

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=False, gridspec_kw={'width_ratios': [1, 1]}, figsize=(12,5))
fig.subplots_adjust(hspace=0.2)
dx = axes[0]
camgeom = source.subarray.tel[3].camera.geometry
disp = CameraDisplay(camgeom, ax = dx)
disp.image = event_container[1053].simulation.tel[3].true_image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
dx.set_title("Extracted muon ring")
cx = axes[1]
camgeom = source.subarray.tel[2].camera.geometry
disp = CameraDisplay(camgeom, ax = cx)
disp.image = event_container[1053].simulation.tel[2].true_image
#disp.image = true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
cx.set_title("Extracted contamination")
#plt.show()
#plt.savefig("/Users/vdk/DistinctRingContamination")

In [ ]:
#len(event_optical_coef)
lets_check

In [ ]:
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray)
calib(event)
image_processor(event)
muon_processor(event)
np.isnan(j.muon.tel[m].efficiency.width)

In [ ]:
not np.isnan(event.muon.tel[1].efficiency.width)

In [ ]:
if not type(efficiency_container[19]) is int:
    print(1)

In [ ]:
for key in efficiency_container:
    print(efficiency_container[key].optical_efficiency)

In [ ]:
calculated = {}
for i,j in enumerate(efficiency_container):
    if not type(j) is int:
        calculated[i] = j
        

In [ ]:
for i in calculated:
    print(i)

In [ ]:
calculated[2]

In [ ]:
np.radians(0.2)

In [ ]:
len(event_container)

In [ ]:
checker

In [ ]:
164/4227

In [ ]:
3/4227

In [ ]:
3/10000

In [ ]:
for k in range(1,10):
    print(k)

In [ ]:
from traitlets.config import Config

image_processor_config = Config(
    {
        "ImageProcessor": {
            "image_cleaner_type": "TailcutsImageCleaner",
            "TailcutsImageCleaner": {
                "picture_threshold_pe": [
                    ("type", "LST_LST_LSTCam", 7.5),
                    ("type", "MST_MST_FlashCam", 8),
                    ("type", "MST_MST_NectarCam", 8),
                    ("type", "SST_ASTRI_CHEC", 7),
                ],
                "boundary_threshold_pe": [
                    ("type", "LST_LST_LSTCam", 5),
                    ("type", "MST_MST_FlashCam", 4),
                    ("type", "MST_MST_NectarCam", 4),
                    ("type", "SST_ASTRI_CHEC", 4),
                ],
            },
        }
    }
)

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

event = next(event_iterator)

image_processor = ImageProcessor(source.subarray, config = image_processor_config)
image_processor(event)

In [ ]:
muon_processor_config = Config(
    {
        "MuonProcessor": {
            "RingQuery": {
                ["intensity_check","np.abs(parameters.intensity_ratio - 1) < 0.1"]
            }
        }
        "MuonIntensityFitter" :{
        "hole_radius_m" : ["type", "LST_*", 0.308]
        }
    }

)

In [ ]:
7/(18295)*100

In [ ]:
14068+4227

In [ ]:
k = 2
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

for i,j in enumerate(event_iterator):
    if i == 223:
        plt.figure(figsize = (9,6))
        camgeom = source.subarray.tel[3].camera.geometry
        title="Muon Ring?"
        disp = CameraDisplay(camgeom,title=title)
        disp.image = j.simulation.tel[2].true_image
        #disp.image = true_image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.show()
        break
        #plt.savefig(f"/Users/vdk/IntensityCut_event{j.index.event_id}")
        

In [ ]:
len(for_hist)

In [ ]:
for_hist = []
check = 0

for i in event_container:
    #print(i)
    #check+=1
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
            #print(check)
            #print("m", m)
            #print("i", i)
            peak_arr = np.empty(1855, dtype = int)
            np.argmax(event_container[i].r0.tel[m].waveform[1], axis = 1, out = peak_arr)
            #print(check,peak_arr)
            for_hist = np.append(for_hist, peak_arr) 
            check += 1

In [ ]:
histplot(for_hist, alpha = 0.8, binwidth = 1, binrange=[0,40])
plt.xlabel("Time sample")
plt.ylabel("Count of pixels with appropriate peaktime")
#plt.title(f"Histogram for peaktime of 40 runs minpixel > {min_pixel}")
#plt.savefig(f"/Users/vdk/Histplot_40runs_PeakTime_minpixel>{min_pixel}")
#plt.title(f"Histogram for peaktime of 40 runs IntensityCut<{int_cut}")
plt.title(f"Histogram for r0 high gain")
#plt.savefig(f"/Users/vdk/Histplot_40runs_IntensityCut>{int_cut}.png")

In [ ]:
test = histplot(for_hist, alpha = 0.8, bins = [i-0.5 for i in range(41)])
plt.title(f"Histogram for r0 low gain")

In [ ]:
hist, bin_edges = np.histogram(for_hist, bins =40)

In [ ]:
bin_edges

In [ ]:
plt.hist(for_hist, bins = [i for i in range(41)])

In [ ]:
min(for_hist)

In [ ]:
len(hist)

In [ ]:
bin_edges

In [ ]:
import pandas as pd

# Generate data on commute times.
size, scale = 1000, 10
commutes = pd.Series(for_hist)

commutes.plot.hist(grid=True, bins=40, rwidth=1,
                   color='#607c8e')
plt.title('Commute Times for 1,000 Commuters')
plt.xlabel('Counts')
plt.ylabel('Commute Time')
plt.grid(axis='y', alpha=0.75)

In [ ]:
len([-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5,13.5,14.5,15.5,16.5,17.5,18.5,19.5,20.5,21.5,22.5,23.5,24.5,25.5,26.5,27.5,28.5,29.5,30.5,31.5,32.5,33.5,34.5,35.5,36.5,37.5,38.5,39.5])

In [ ]:
for i in range(0,41):
    print(i-0.5)

In [ ]:
arr = [1,6,3,2,5,5,6,4,4,4,3,3,3,2,6]
histplot(arr)

In [ ]:
for i in for_hist:
    print(i)

In [ ]:
np.count(for_hist,1)

In [ ]:
other = for_hist.tolist()

In [ ]:
other.count(38)

In [ ]:
np.char.count(arr,'1')

In [ ]:
for_hist = []
check = 0
time_sample = 24

for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
            max_val = np.amax(event_container[i].r1.tel[m].waveform[time_sample])
            event_list = event_container[i].r1.tel[m].waveform[time_sample].tolist()
            #print("max", max_val)
            #print("events", event_container[i].r1.tel[m].waveform.shape)
            counts = event_list.count(max_val)
            print("counts = ", counts)
            
            

In [ ]:
for_hist = []
check = 0

for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
            peak_arr = np.empty(1855, dtype = int)
            np.argmax(event_container[i].r1.tel[m].waveform, axis = 1, out = peak_arr)
            print(check,peak_arr)
            for_hist = np.append(for_hist, peak_arr) 
            check += 1

winner = np.argwhere(event_container[i].r1.tel[m].waveform == np.amax(event_container[i].r1.tel[m].waveform))

In [ ]:
winner = np.argwhere(event_container[i].r1.tel[m].waveform == np.amax(event_container[i].r1.tel[m].waveform))

In [ ]:
winner

In [ ]:
k = 22
event_number = 86
teln = 3
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

for i,j in enumerate(event_iterator):
    if i == event_number:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        event = j
        break


# check only pixels where signal is less then some value
# mean value for all pixels sum over the samples is 6200
#work_arr = event.r0.tel[1].waveform[0].copy()
#work_arr[np.sum(work_arr, axis = 1) > 10000] = 0
#teln = 2
#print(np.count_nonzero(work_arr))

plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[1].camera.geometry
title="Ring"
disp = CameraDisplay(camgeom,title=title, norm = 'lin', autoscale = False, allow_pick = True)

disp.image = event.r0.tel[teln].waveform[1].sum(axis=1)
#disp.image = event.dl0.tel[teln].waveform[:,11]
#disp.image = working_arr
#disp.image = event.simulation.tel[teln].true_image
#disp.image = cont_arr
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
print("print", disp.pixels)
#plt.savefig(f"/Users/vdk/IntensityCut_event{j.index.event_id}")

In [ ]:
event_container[22086].r1.tel[2].selected_gain_channel

In [ ]:
for i in event_container[22086].simulation.shower.energy

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.2
min_pixel = 250

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.3"],
                                                        ["ring_completeness", "parameters.completeness > 0.3"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})
#k = 1
checker = 0
event_container_protons = []

for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1:
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(j)
            image_processor(j)
            muon_processor(j)
            for m in j.muon.tel.keys():
                if not np.isnan(j.muon.tel[m].efficiency.width):
                    event_container_protons.append(j)
    if len(event_container_protons) > 43:
        break
        
            
            


In [ ]:
len(event_container_protons)

In [ ]:
for i,j in enumerate(event_container_protons):
    for m in j.trigger.tel.keys():
        if not np.isnan(j.muon.tel[m].efficiency.width):
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                #title=f"event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                #title=f"event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggerTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = j.simulation.tel[m].true_image
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                plt.show()
                
plt.close()

In [ ]:
for_hist_muons = []
check = 0
bad_muons = [2277, 8234,  8448, 9404,  9433, 13088, 18054, 20351, 27176, 27374, 28000, 28328, 35298, 35473]

for i in event_container:
    #print(i)
    #check+=1
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width) and i not in bad_muons:
            #print(check)
            #print("m", m)
            #print("i", i)
            peak_arr = np.empty(1855, dtype = int)
            np.argmax(event_container[i].r0.tel[m].waveform[1], axis = 1, out = peak_arr)
            #print(check,peak_arr)
            for_hist_muons = np.append(for_hist_muons, peak_arr) 
            check += 1

In [ ]:
len(for_hist_muons)/1855

In [ ]:
for_hist_protons = []
check = 0
bad_muons = [2277, 8234,  8448, 9404,  9433, 13088, 18054, 20351, 27176, 27374, 28000, 28328, 35298, 35473]

for i,j in enumerate(event_container_protons[2:33]):
    #print(i)
    #check+=1
    for m in j.trigger.tel.keys():
        if not np.isnan(j.muon.tel[m].efficiency.width):
            #print(check)
            #print("m", m)
            #print("i", i)
            peak_arr = np.empty(1855, dtype = int)
            np.argmax(j.r0.tel[m].waveform[1], axis = 1, out = peak_arr)
            #print(check,peak_arr)
            for_hist_protons = np.append(for_hist, peak_arr) 
            check += 1

In [ ]:
len(for_hist_protons)/1855

In [ ]:
stack = np.vstack((for_hist_muons,for_hist_protons))

In [ ]:
len(stack[0])

In [ ]:
for i in stack:
    histplot(i, bins = 40, alpha = 0.1)

In [ ]:
k = 13
if k in [13,14,15]:
    print(True)

In [ ]:
muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
for k in range(1,5):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1: 
            #print('i am here')
            event = j
            image_processor = ImageProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(event)
            image_processor(event)
            for m in event.muon.tel.keys():
                print('i am here')
                print(event.muon.tel[m].parameters)
            #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            #image_processor = ImageProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray)
            #muon_processor(event)
            #for m in j.muon.tel.keys():
            #    if not np.isnan(j.muon.tel[m].efficiency.width):
            #        event_container[keks+i] = j

In [ ]:
print(event.muon.tel[1].parameters)

In [ ]:
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
            print(event_container[i].muon.tel[m].efficiency.optical_efficiency)

In [ ]:
1713.87/60

In [ ]:
1713/36500

In [ ]:
0.21164505592802652/26

In [ ]:
cleaned_event_container[19358].muon.tel[m].parameters.completeness

In [ ]:
0.02824719946917591/0.18

In [ ]:
0.02824719946917591/0.18774292339345722/48**0.5

In [ ]:
0.0267/0.1812/33**0.5


In [ ]:
#histplot(optical_coef, bins = 10)

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
#int_cut = 0.3
min_pixel = 150

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
test_event_container = {}
t1 = time.perf_counter(), time.process_time()

#for k in range(1,41):
filename = f'/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput//simtel_run501_muon_telescope_transmission_0.8.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
keks = k*1000
for i,j in enumerate(event_iterator):
    checker += 1
    #if len(j.trigger.tels_with_trigger) > 1:            
    image_processor = ImageProcessor(source.subarray)
    #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(
        image_extractor_type="GlobalPeakWindowSum",
        subarray = source.subarray)
    calib(j)
    image_processor(j)
    muon_processor(j)
    for m in j.muon.tel.keys():
        if not np.isnan(j.muon.tel[m].efficiency.width):
            test_event_container[keks+i] = j
t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
test_event_container

In [ ]:
# PROCESSING OF REGULAR IMAGES 
##
#
# For all telescopes
#
#
#

lets_check = 0
opt_eff_cleaned = []
for i in test_event_container:
    for m in test_event_container[i].trigger.tel.keys():
        #if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                print("n_pixels ", test_event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                print("MSE ", test_event_container[i].muon.tel[m].parameters.mean_squared_error)                
                print("Muon parameters ", test_event_container[i].muon.tel[m].parameters)
                print("Muon other parameters ", test_event_container[i].dl1.tel[m].parameters.hillas)                
                print("intensity_check ", abs(test_event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                #if event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.1:
                #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                lets_check+=1
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"Event_{i}_optEff{round(test_event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(test_event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(test_event_container[i].trigger.tel.keys())}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = test_event_container[i].simulation.tel[m].true_image
                #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                plt.show()
                #plt.savefig(f"/Users/vdk/ImagePurgatorium/MuonsWithoutCleaning_intensitycut<0.3_event{i}.png")
                #opt_eff_cleaned.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)
                    
plt.close()

In [ ]:
test_event_container[1854000].simulation.shower.energy

In [ ]:
source.subarray.peek()

In [ ]:
energies = []
for i in event_container:
    energies.append(event_container[i].simulation.shower.energy)
    

In [ ]:
min(energies)

In [ ]:
test_energies = []
for i in test_event_container:
    test_energies.append(test_event_container[i].simulation.shower.energy)

In [ ]:
test_energies

In [ ]:
40000/20

In [ ]:
j

In [ ]:
j.r0.tel[1].waveform

In [ ]:
### NEW CLEANING NEW CLEANING NEW CLEANING 
####
### TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST
###
# Algorithm to process events and calibrate with new cleaning algo
###
int_cut = 0.2
min_pixel = 150

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
cleaned_event_container = {}
t1 = time.perf_counter(), time.process_time()
for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1: 
            event = j
            image_processor = ImageProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
            muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            for tels in event.trigger.tels_with_trigger:
                pixel_mask = []
                for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                    if np.argmax(l) in range(16,40): 
                        pixel_mask.append(True)
                    else:
                        pixel_mask.append(False)
                working_wave = event.r1.tel[tels].waveform.copy()
                for k,l in enumerate(working_wave):
                    if pixel_mask[k]:
                        #working_wave[k] = np.zeros(40)     
                        working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                event.r1.tel[tels].waveform = working_wave
                pre = np.mean(event.r1.tel[tels].waveform, axis = 1).sum()
                #print("PRE", np.mean(event.r1.tel[tels].waveform, axis = 1))
                calib(event)
                image_processor(event)
                muon_processor(event)
                #print("POST", np.mean(event.r1.tel[tels].waveform, axis = 1 ))
                #print("____________________________________")
                post = np.mean(event.r1.tel[tels].waveform, axis = 1).sum()
                if not pre == post:
                    print("1")
                else:
                    print("0")
        for m in j.muon.tel.keys():
            if not np.isnan(j.muon.tel[m].efficiency.width):
                cleaned_event_container[keks+i] = j
        t2 = time.perf_counter(), time.process_time()
        




            
            
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")      



In [ ]:
event.trigger.tels_with_trigger[0]

In [ ]:
###
# Calculate triggered telescopes
###
int_cut = 0.4
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

tels_1_2 = []
tels_1_3 = []
tels_1_2_3 = []
tels = []

for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1:            
            tels.append(j.trigger.tels_with_trigger)
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
test = [[1,2],[1,3], [1,2]]

In [ ]:
tels.count([1, 2].all())

In [ ]:
fi_tw = 0
fi_th = 0
fi_tw_th = 0
tw_th = 0

for i in tels:
    print(i)
    if (i == [1,2]).any():
        fi_tw += 1

In [ ]:
tels

In [ ]:
arr1 = [1,2,3]
arr2 = [2,3,4]

arr1 == arr2

In [ ]:
two = []
three = []
for i in tels:
    if len(i) == 2:
        two.append(i)
    else:
        three.append(i)

In [ ]:
two = np.array(two)

In [ ]:
len(three)

In [ ]:
two.count([1,2])

In [ ]:
np.count_nonzero(two==[1,2])

In [ ]:
np.count_nonzero(two==[1,3])

In [ ]:
np.count_nonzero(two==[2,3])

In [ ]:
fi_tw = 0
fi_th = 0
fi_tw_th = 0
tw_th = 0
for i in two:
    if (i == [1,2]).all():
        fi_tw +=1
    elif (i == [2,3]).all():
        tw_th +=1
    elif (i == [1,3]).all():
        fi_th +=1

print("1-2" , fi_tw)
print("2-3" , tw_th)
print("1-3" , fi_th)

In [ ]:
tw_th+ fi_th + fi_tw + len(three)

In [ ]:
energies = []
for i in event_container:
    energies.append(event_container[i].simulation.shower.energy.value)

In [ ]:
energies

In [ ]:
event_container